In [16]:
%reset

In [17]:
import polars as pl
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_tbl_rows(-1)

df_problems = pl.read_parquet('sampled_codeforces_data.parquet')
# df.write_parquet('codeforces_data.parquet')

In [18]:
df_problems.head(10)

shape: (10, 6)
┌─────────────────┬─────────────────┬─────────────────┬────────────────┬──────────┬────────────────┐
│ name            ┆ description     ┆ public_tests    ┆ private_tests  ┆ cf_index ┆ cf_tags        │
│ ---             ┆ ---             ┆ ---             ┆ ---            ┆ ---      ┆ ---            │
│ str             ┆ str             ┆ struct[2]       ┆ struct[2]      ┆ str      ┆ list[str]      │
╞═════════════════╪═════════════════╪═════════════════╪════════════════╪══════════╪════════════════╡
│ 500_C. New Year ┆ New Year is     ┆ {["3 5          ┆ {["50 50       ┆ C        ┆ ["constructive │
│ Book Reading    ┆ coming, and     ┆ 1 2 3           ┆ 75 71 23 37 28 ┆          ┆ algorithms",   │
│                 ┆ Jaehyun decided ┆ 1 3 2 3 1       ┆ 23 69 75 5 62  ┆          ┆ "greedy", …    │
│                 ┆ to read many    ┆ "],["12         ┆ 3 11 96 100 13 ┆          ┆ "math"]        │
│                 ┆ books during    ┆ "]}             ┆ 50 57 51 8 90  ┆          ┆                │
│                 ┆ 2015, unlike    ┆                 ┆ 4 6 84 27 11   ┆          ┆                │
│                 ┆ this year. He   ┆                 ┆ 89 95 81 10 62 ┆          ┆                │
│                 ┆ has n books     ┆                 ┆ 48 52 69 87 97 ┆          ┆                │
│                 ┆ numbered by     ┆                 ┆ 95 30 74 21 42 ┆          ┆                │
│                 ┆ integers from 1 ┆                 ┆ 36 64 31 80 81 ┆          ┆                │
│                 ┆ to n. The       ┆                 ┆ 50 56 53 33 99 ┆          ┆                │
│                 ┆ weight of the   ┆                 ┆ 26 30 5 33 35  ┆          ┆                │
│                 ┆ i-th (1 ≤ i ≤   ┆                 ┆ 29 6 15 36 17  ┆          ┆                │
│                 ┆ n) book is wi.  ┆                 ┆ 32 16 14 1 29  ┆          ┆                │
│                 ┆                 ┆                 ┆ 34 …           ┆          ┆                │
│                 ┆ As Jaehyun's …  ┆                 ┆                ┆          ┆                │
│ 643_B. Bear and ┆ Bearland has n  ┆ {["1000 999     ┆ {["1000 1001   ┆ B        ┆ ["constructive │
│ Two Paths       ┆ cities,         ┆ 10 20 30 40     ┆ 217 636 713    ┆          ┆ algorithms",   │
│                 ┆ numbered 1      ┆ ", "7 11        ┆ 516            ┆          ┆ "graphs"]      │
│                 ┆ through n.      ┆ 2 4 7 3         ┆ ", "1000 1998  ┆          ┆                │
│                 ┆ Cities are      ┆ "],["-1", "2 7  ┆ 833 681 19 233 ┆          ┆                │
│                 ┆ connected via   ┆ 1 5 6 3 4       ┆ ", … "4 5      ┆          ┆                │
│                 ┆ bidirectional   ┆ 7 2 1 5 6 4 3   ┆ 1 2 3 4        ┆          ┆                │
│                 ┆ roads. Each     ┆ "]}             ┆ "],["217 713 1 ┆          ┆                │
│                 ┆ road connects   ┆                 ┆ 2 3 4 5 6 7 8  ┆          ┆                │
│                 ┆ two distinct    ┆                 ┆ 9 10 11 12 13  ┆          ┆                │
│                 ┆ cities. No two  ┆                 ┆ 14 15 16 17 18 ┆          ┆                │
│                 ┆ roads connect   ┆                 ┆ 19 20 21 22 23 ┆          ┆                │
│                 ┆ the same pair   ┆                 ┆ 24 25 26 27 28 ┆          ┆                │
│                 ┆ of cities.      ┆                 ┆ 29 30 31 32 33 ┆          ┆                │
│                 ┆                 ┆                 ┆ 34 35 36 37 38 ┆          ┆                │
│                 ┆ Bear Limak was  ┆                 ┆ 39 40 …        ┆          ┆                │
│                 ┆ once in…        ┆                 ┆                ┆          ┆                │
│ 774_D. Lie or   ┆ Vasya has a     ┆ {["4 2 4        ┆ {["5 1 3       ┆ D        ┆ ["*special",   │
│ Truth           ┆ sequence of     ┆ 1 1 1 1 

In [19]:
df_problems["public_tests"][1]

{'input': ['1000 999\n10 20 30 40\n', '7 11\n2 4 7 3\n'],
 'output': ['-1', '2 7 1 5 6 3 4 \n7 2 1 5 6 4 3 ']}

In [20]:
df_problems.height

120

In [21]:
# Create df for statistics: pass_k (k_tries + 1), k_tries (number of debugging processes), exec_time, total_llm_calls, total_input_tokens, total_output_tokens, total_tokens, total_costs_usd, number_given_pub_tests, algorithm_name

# Step 1: Create a new empty DataFrame with columns copied from the existing df

schema = {
    "name": pl.Utf8,
    "description": pl.Utf8,
    "public_tests": pl.Struct([pl.Field("input", pl.List(pl.Utf8)), pl.Field("output", pl.List(pl.Utf8))]),
    "private_tests": pl.Struct([pl.Field("input", pl.List(pl.Utf8)), pl.Field("output", pl.List(pl.Utf8))]),
    "cf_index": pl.Utf8,
    "cf_tags": pl.List(pl.Utf8),
    "solution_algorithm_name": pl.Utf8,
    "pass_k": pl.Int64,
    "k_tries": pl.Int64,
    "exec_time_sec": pl.Float64,
    "total_llm_calls": pl.Int64,
    "total_input_tokens": pl.Int64,
    "total_output_tokens": pl.Int64,
    "total_tokens": pl.Int64,
    "total_costs_usd": pl.Float64,
    "k_debug": pl.Int64,
    "t_plan": pl.Int64,
    "programming_language": pl.Utf8,
    "code_solution": pl.Utf8,
    "is_solved": pl.Boolean,
    "number_given_pub_tests": pl.Int64,
    "total_number_test_cases": pl.Int64,
    "number_failed_test_cases": pl.Int64,
}

df_statistics = pl.DataFrame(schema=schema)

In [22]:
from langchain_core.runnables import RunnableConfig
import time
from langchain_community.callbacks import get_openai_callback
from agent.graph.utils.state import TestCase
from typing import Dict, List
from agent.graph.graph import graph
from dotenv import load_dotenv
from tqdm import tqdm


# Load API keys from .env
load_dotenv()

def convert_to_test_case(tests_dict: Dict, is_public: bool = False) -> List[TestCase]:
    return [
        TestCase(input=inp, expected_output=out, is_public=is_public)
        for inp, out in zip(tests_dict.get('input', []), tests_dict.get('output', []))
    ]

# Setup variables K, T
programming_language = 'python'
K = 3
T = 3
config = RunnableConfig(recursion_limit=100)

output_file_path = "/Users/maksim/Documents/VSE/Bachelor's thesis/self-corrective-coding-agent/data/experiment_results.parquet"
failed_indexes_path = "/Users/maksim/Documents/VSE/Bachelor's thesis/self-corrective-coding-agent/data/failed_indexes.csv"

failed_indexes = []

In [ ]:
# Start iteration over problems

for i in tqdm(range(df_problems.height), desc="Processing Problems", unit="problem", ncols=80):
    input_state = {
        "original_problem": df_problems['description'][i],
        "public_tests": convert_to_test_case(df_problems['public_tests'][i]),
        "private_tests": convert_to_test_case(df_problems['private_tests'][i]),
        "programming_language": programming_language,
        "k_debug": K,
        "t_plan": T,
    }
    
    # Track starting time
    start_time = time.time()
    try:
        with get_openai_callback() as cb:
            final_state = await graph.ainvoke(input_state, config=config)
    except Exception as e:
        final_state = None
        print(f"Error with problem index {i}: {e}")
        failed_indexes.append(i)
        continue
    
    # Calculate execution time
    end_time = time.time()
    execution_time = round(end_time - start_time, 2)
    
    # Append callbacks statistics and time to the statistics dataframe
    plans_sorted = final_state.get("gen_plans").plans
    t = final_state.get("t_current") if final_state.get("t_current") < T else T-1
    solution_algorithm_name = plans_sorted[t].algorithm_name
    
    row_data = {
        "name": df_problems['name'][i],
        "description": df_problems['description'][i],
        "public_tests": df_problems['public_tests'][i],
        "private_tests": df_problems['private_tests'][i],
        "cf_index": df_problems['cf_index'][i],
        "cf_tags": df_problems['cf_tags'][i],
        "solution_algorithm_name": solution_algorithm_name,
        "pass_k": final_state.get("k_tries") + 1,
        "k_tries": final_state.get("k_tries"),
        "exec_time_sec": execution_time,
        "total_llm_calls": cb.successful_requests,
        "total_input_tokens": cb.prompt_tokens,
        "total_output_tokens": cb.completion_tokens,
        "total_tokens": cb.total_tokens,
        "total_costs_usd": cb.total_cost,
        "k_debug": K,
        "t_plan": T,
        "programming_language": programming_language,
        "code_solution": final_state.get('code'),
        "is_solved": final_state.get('test_evaluation_result').all_tests_passed,
        "number_given_pub_tests": len(df_problems['public_tests'][i]["input"]),
        "total_number_test_cases": len(final_state.get('public_tests')) + len(final_state.get('private_tests')),
        "number_failed_test_cases": len(final_state.get('test_evaluation_result').debug_info.failed_cases) if not final_state.get('test_evaluation_result').all_tests_passed else 0,
    }
    
    row_df = pl.DataFrame([row_data])
    df_statistics = pl.concat([df_statistics, row_df], how="vertical")
    df_statistics.write_parquet(output_file_path)
    time.sleep(2)
    
failed_df = pl.DataFrame({"failed_problem_indexes": failed_indexes})
failed_df.write_csv(failed_indexes_path)


Processing Problems:   0%|                         | 0/120 [00:00<?, ?problem/s]